# Planet Tasking API Monitoring Tasking Orders

---

## Introduction

---

This tutorial is an introduction on how to create monitoring tasking orders using [Planet](https://www.planet.com)'s Tasking API. It provides code samples on how to write simple Python code to do this.

The API reference documentation can be found at https://developers.planet.com/docs/tasking

### Requirements

---

#### Software & Modules

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Familiarity with basic REST API concepts and usage is also assumed.

We'll be using a **"Jupyter Notebook"** (aka Python Notebook) to run through the examples.
To learn more about and get started with using Jupyter, visit: [Jupyter](https://jupyter.org/) and [IPython](https://ipython.org/). 

For the best experience, download this notebook and run it on your system, and make sure to install the modules listed below first. You can also copy the examples' code to a separate Python files an run them directly with Python on your system if you prefer.

#### Planet API Key

You should have an account on the Planet Platform to access the Tasking API. You may retrieve your API key from your [account page](https://www.planet.com/account/), or from the "API Tab" in [Planet Explorer](https://www.planet.com/explorer).

## Overview

---

### The basic workflow

1. Create a monitoring tasking order
1. Check the status of the tasking order

### API Endpoints

This tutorial will cover the following API ***endpoint***:

* [`/order`](https://api.planet.com/tasking/v2/order/)

## Basic Setup

---

Before interacting with the Planet Tasking API using Python, we will set up our environment with some useful modules and helper functions.

* We'll configure *authentication* to the Planet Tasking API
* We'll use the `requests` Python module to make HTTP communication easier. 
* We'll use the `json` Python module to help us work with JSON responses from the API.
* We'll use the `pytz` Python module to define the time frame for the order that we will be creating.
* We'll create a function called `p` that will print Python dictionaries nicely.

Then we'll be ready to make our first call to the Planet Tasking API by hitting the base endpoint at `https://api.planet.com/tasking/v2`. 

Let's start by configuring authentication:

### Authentication

Authentication with the Planet Tasking API can be achieved using a valid Planet **API key**.

You can *export* your API Key as an environment variable on your system:

`export PL_API_KEY="YOUR API KEY HERE"`

Or add the variable to your path, etc.

To start our Python code, we'll setup an API Key variable from an environment variable to use with our requests:

In [ ]:
# Import the os module in order to access environment variables
import os

#If you are running this notebook outside of the docker environment that comes with the repo, you can uncomment the next line to provide your API key
#os.environ['PL_API_KEY']=input('Please provide your API Key')

# Setup the API Key from the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

### Helper Modules and Functions

In [ ]:
# Import helper modules
import json
import requests
import pytz
from time import sleep
from datetime import datetime, timedelta

In [ ]:
# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

In [ ]:
# Setup Planet Tasking PLANET_API_HOST
TASKING_API_URL = "https://api.planet.com/tasking/v2"

# Setup the session
session = requests.Session()

# Authenticate
session.headers.update({
    'Authorization': f'api-key {PLANET_API_KEY}',
    'Content-Type': 'application/json'
})

In [ ]:
## 1 | Compose the monitoring tasking order

We want to create a monitoring tasking order that can be set up to take images of the same location at a defined cadence, in this example once per week for a month. To keep things simple we are going to create a Point order, which takes a single latitude/longitude coordinate pair. Since this is your monitoring order, you need to provide the details of what the tasking order is called and the coordinates. 

To make things easier, we will default the start time to start tomorrow with the aim of taking four images if we stick to the weekly cadence. Of course, feel free to change this to suit your needs, but if you do, take note that all times should be in UTC format.

Monitoring Tasking Orders are composed slightly different than other types of Tasking Order in that they do not require an *end time* to be defined, in fact the end time is ignored even if it is provided. Instead it requires a **rrule** string to be defined that will set the frequency, the interval and the count, which, when taken together, define the overall cadence pattern for the monitoring order. The only required field in the rrule string is the **count**, although in this example we are also defining the frequency, the interval and the day that each cadence event should begin using the **BYDAY** parameter. More information on the rrule and how it is used for Tasking Orders can be found in the [Planet Developers Portal](https://developers.planet.com/docs/tasking/examples/monitoring)


In [ ]:
# Define the name and coordinates for the order
name=input("Give the order a name")
latitude=float(input("Provide the latitude"))
longitude=float(input("Provide the longitude"))

# Because the geometry is GeoJSON, the coordinates must be longitude,latitude. 
# It is also necessary to set the scheduling_type to "MONITORING"
order = {
    'name': name,
    'geometry': {
        'type': 'Point',
        'coordinates': [
            longitude,
            latitude
        ]
    },
    "scheduling_type": "MONITORING"
}

# Set a start time
tomorrow = datetime.now(pytz.utc) + timedelta(days=1)

# define the rrule
rrule="FREQ=WEEKLY;COUNT=4;INTERVAL=1;BYDAY=MO"

monitoring_parameters = {
    'start_time': tomorrow.isoformat(),
    'rrule': rrule
}

# Add the monitoring parameters
order.update(monitoring_parameters)

In [ ]:
#View the payload before posting
p(order)

In [ ]:
# The creation of an order is a POST request to the /orders endpoint
res = session.request('POST', TASKING_API_URL + '/orders/', json=order)

if res.status_code == 403:
    print('Your PLANET_API_KEY is valid, but you are not authorized.')
elif res.status_code == 401:
    print('Your PLANET_API_KEY is incorrect')
elif res.status_code == 201:
    print('Your order was created successfully')
else:
    print(f'Received status code {res.status_code} from the API. Please contact support.')

# View the response
p(res.json())

**Congratulations!** You just created a monitoring tasking order using the Planet Tasking API. Depending on the parameters that you provided, a satellite will be attempting to take an image over your given coordinates in the near future.

## 2 | Check the status of the monitoring order

To see the status of the new monitoring tasking order, the tasking order id is required. Depending on the tasking order, it can take some time for the status of the tasking order to change, and so you may need to come back to this section once some time has elapsed before changes to the tasking order can be seen. It is recommended to run the next part of this notebook to extract the ID of the newly created order and save that for later use.

In [ ]:
# Get the response JSON and extract the ID of the order
response = res.json()
new_order_id = response["id"]
p(new_order_id)

In [ ]:
def check_order_status(order_id):
    # Make a GET request with the order_id concatenated to the end of the /orders url; e.g. https://api.planet.com/tasking/v2/orders/<ORDER_ID>
    res = session.request('GET', TASKING_API_URL + '/orders/' + order_id)

    if res.status_code == 403:
        print('Your PLANET_API_KEYPLANET_API_KEY is valid, but you are not authorized to view this order.')
    elif res.status_code == 401:
        print('Your PLANET_API_KEYPLANET_API_KEY is incorrect')
    elif res.status_code == 404:
        print(f'Your order ({order_id}) does not exist')
    elif res.status_code != 200:
        print(f'Received status code {res.status_code} from the API. Please contact support.')
    else:
        order = res.json()
        p(res.json())
        print(f'Your order is {order["status"]} with {order["capture_status_published_count"]} published captures '
                f'and {order["capture_assessment_success_count"]} successful captures')

In [ ]:
check_order_status(new_order_id)